In [ ]:
!pip install gymnasium

In [ ]:
import numpy as np
import gymnasium as gym
from gymnasium import spaces

class RiverSwimEnv(gym.Env):
    def __init__(self, nS=6):
        super(RiverSwimEnv, self).__init__()
        self.nS = nS
        self.nA = 2  # LEFT = 0, RIGHT = 1
        self.state = 0
        self.steps_taken = 0
        self.max_steps = 20

        self.action_space = spaces.Discrete(self.nA)
        self.observation_space = spaces.Discrete(self.nS)

        # Define transition probabilities and rewards
        self.P = self._init_dynamics()

    def _init_dynamics(self):
        P = {}
        for s in range(self.nS):
            P[s] = {a: [] for a in range(self.nA)}

        # LEFT transitions
        for s in range(self.nS):
            P[s][0] = [(1.0, max(0, s-1), 5/1000 if s == 0 else 0, False)]

        # RIGHT transitions
        P[0][1] = [(0.3, 0, 0, False), (0.7, 1, 0, False)]
        for s in range(1, self.nS - 1):
            P[s][1] = [
                (0.1, max(0, s-1), 0, False),
                (0.6, s, 0, False),
                (0.3, min(self.nS-1, s+1), 0, False)
            ]
        P[self.nS-1][1] = [(0.7, self.nS-1, 1, False), (0.3, self.nS-2, 0, False)]

        return P

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.state = 0
        self.steps_taken = 0
        return self.state, {}

    def step(self, action):
        transitions = self.P[self.state][action]
        i = self.np_random.choice(len(transitions), p=[t[0] for t in transitions])
        p, next_state, reward, _ = transitions[i]
        self.state = next_state
        self.steps_taken += 1

        # Check if max steps reached
        done = self.steps_taken >= self.max_steps

        return next_state, reward, done, False, {}

    def render(self):
        print(f"Current state: {self.state}")

# Register the environment
gym.register(
    id='RiverSwim-v0',
    entry_point='__main__:RiverSwimEnv',
    max_episode_steps=20,
)

# Create the environment
env = gym.make('RiverSwim-v0')

/usr/local/lib/python3.10/dist-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment RiverSwim-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.9 MB/s eta 0:00:00


In [ ]:
!pip install optuna plotly kaleido

In [ ]:
!pip install stable_baselines3

In [ ]:
import optuna
import numpy as np
import gymnasium as gym
from gymnasium import spaces
from stable_baselines3 import PPO, A2C
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy

def optimize_a2c(trial):
    # Define the hyperparameters to optimize
    #n_steps = trial.suggest_int("n_steps", 32, 2048)
    n_steps = 1024
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-2)

    # Create the vectorized environment
    env = make_vec_env('RiverSwim-v0', n_envs=1)

    # Create and train the model
    model = A2C(
        "MlpPolicy",
        env,
        n_steps=n_steps,
        learning_rate=learning_rate,
        verbose=0
    )

    model.learn(total_timesteps=200000)

    # Evaluate the model
    mean_reward, _ = evaluate_policy(model, env, n_eval_episodes=10)

    return mean_reward

def main():
    study = optuna.create_study(direction="maximize")
    study.optimize(optimize_a2c, n_trials=100)

    print("Best trial:")
    trial = study.best_trial

    print("Value: ", trial.value)
    print("Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

    # Train a final model with the best hyperparameters
    env = make_vec_env('RiverSwim-v0', n_envs=1)
    best_model = A2C("MlpPolicy", env, **trial.params, verbose=1)
    best_model.learn(total_timesteps=200000)

    # Save the best model
    best_model.save("a2c_deepsea_best")

    # Final evaluation
    final_env = gym.make('DeepSea-v0')
    mean_reward, std_reward = evaluate_policy(best_model, final_env, n_eval_episodes=100)
    print(f"Final mean reward: {mean_reward:.2f} +/- {std_reward:.2f}")

if __name__ == "__main__":
    main()

[I 2024-10-05 22:09:31,403] A new study created in memory with name: no-name-e7dbfd18-fab5-4e69-a937-2aaa833c51a0
<ipython-input-10-26ad73cf07ba>:13: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-2)
/usr/local/lib/python3.10/dist-packages/gymnasium/envs/registration.py:788: UserWarning: WARN: The environment is being initialised with render_mode='rgb_array' that is not in the possible render_modes ([]).
  logger.warn(
[I 2024-10-05 22:13:07,078] Trial 0 finished with value: 2.4 and parameters: {'learning_rate': 0.0032172072865183283}. Best is trial 0 with value: 2.4.
<ipython-input-10-26ad73cf07ba>:13: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0.